## 연습문제 1
#### 투수들의 기록중에서 평균자책점(ERA), 투구인닝(IP), 탈삼진(SO) 기록을 찾아서 Pitcher_stats 란 테이블을 만들고, Eagles 테이블과 Join 하여 백넘버, 선수명, 포지션, 투구인닝, 평균자책점, 탈삼진 필드를 갖는 데이터 프레임을 만들어서 Join 한 결과를 입력하고, 그 결과를 보이시오.

In [75]:
import sqlite3

In [77]:
import pandas as pd

In [76]:
conn = sqlite3.connect('./test.db') 
'''
conn = sqlite3.connect(':memory:')    # 메모리 DB 접속(일회성)
'''

"\nconn = sqlite3.connect(':memory:')    # 메모리 DB 접속(일회성)\n"

In [70]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Pitcher_stats \
    (id INT NOT NULL, \
     player TEXT, \
     era REAL, \
     ip REAL, \
     so INT, \
     PRIMARY KEY(id));')

In [78]:
pstats = pd.read_csv('./pstats.csv', encoding='EUC-KR')
pstats

,player,era,ip,so
0,장민재,4.81,91.2,84
1,안영명,3.43,44.2,35
2,김범수,5.67,81.0,65


In [72]:
cur = conn.cursor()
sql = 'INSERT INTO Pitcher_stats VALUES (?, ?, ?, ?, ?);'
for i in range(3):
    cur.execute(sql, (i+1, 
                      pstats.iloc[i,0],
                      float(pstats.iloc[i,1]), 
                      float(pstats.iloc[i,2]),
                      int(pstats.iloc[i,3])))
conn.commit()

In [79]:
sql = "SELECT e.back_no, e.name, e.position, \
           p.era, p.ip, p.so \
           FROM Eagles AS e JOIN Pitcher_stats AS p \
           ON e.name like p.player;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnPitcher = ['등번호', '선수명', '포지션', '평균자책점', '투구인닝', '탈삼진']
pitcher_df = pd.DataFrame(columns = columnPitcher)
for row in rows:
    pitcher_df = pitcher_df.append(pd.DataFrame([list(row)], columns = columnPitcher), 
                                   ignore_index=True)
pitcher_df

,등번호,선수명,포지션,평균자책점,투구인닝,탈삼진
0,17,김범수,투수,5.67,81.0,65
1,38,안영명,투수,3.43,44.2,35
2,36,장민재,투수,4.81,91.2,84


In [80]:
conn.close()